In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# We will import the necessary libraries here. You can import any librariese you need for your beautiful factors!
from in_house_validation import utils, factor_design, factor_backtest
import numpy as np
import pandas as pd
from tqdm import tqdm
import queue
from scipy.stats import stats

# Load data

In [25]:
df_train = pd.read_csv('research_train_set.csv')
df_test = pd.read_csv('research_test_set.csv')

In [26]:
col2index_map = utils.load_json('./in_house_validation/col2index_map.json')

In [27]:
df_train_dic_sorted = utils.load_json('./in_house_validation/df_train_dic_sorted.json')
df_test_dic_sorted = utils.load_json('./in_house_validation/test_data_dic_sorted.json')

In [28]:
existed_factor_train = utils.load_json('./in_house_validation/existed_factors.json')
existed_factor_test = utils.load_json('./in_house_validation/existed_factor_test.json')

In [29]:
factor_performance_train = utils.load_json('./in_house_validation/factor_performance.json')
factor_performance_test = utils.load_json('./in_house_validation/factor_performance_test.json')

# New factor here

In [30]:
def s1_imbalance_signed_pow_opt(current_data: dict, hist_list=[]) -> np.ndarray:
    '''
    This will be the main function to design your factors for the competition. Please
    define only one factor here each time. We provide you with:

    Current_data: a dictionary in the format of {column_name: column_value}, where column_name is from the original
    dataframe

    hist_list: A list for you to save the previous factor values (optional). For instance,
    if you are calculating a 100-day Moving Average (MA), then you can save the first calculated
    MA in hist_list, and then for the next MA calculation, you can use the saved ones.
    '''
    ###################### ADD YOUR CODE HERE FOR FACTORS DESIGN ######################
    # convert the current_data to your choice of numpy or pandas dataframe
    # current_data = pd.DataFrame(current_data)
    current_data = np.array(list(current_data.values()),
                            dtype=float).T  # this is faster than pd.DataFrame(current_data).values
    res = (current_data[:, col2index_map['bid_size']] - current_data[:, col2index_map['ask_size']]) / \
          (current_data[:, col2index_map['bid_size']] + current_data[:, col2index_map['ask_size']])
    ############################## NAN/Inf handling ######################################
    # if you have nan in your factor value, please fill it reasonably
    # res = np.nan_to_num(res) # this is slow because it also checks for inf.
    # res = np.where(np.isnan(res), 0, res)  # this is slightly faster than np.nan_to_num
    res[np.isnan(res)] = 0  # this is the fastest way to fill nan
    ############################## END OF YOUR CODE ##############################
    return np.sign(-res) * np.abs(
        res) ** 1.4242424242424243  # The return value MUSE BE a numpy array, with no NaN value
    # reason: s1_imbalance have most information in sign, sign can be think of as sign*abs**0. we optimize the power and get 1.424 as the best
    ####################################################################################


In [31]:
def past_10s_ret(current_data: dict, hist_list=[]):
    current_data = np.array(list(current_data.values()),
                            dtype=float).T  # this is faster than pd.DataFrame(current_data).values
    current_time = current_data[0, col2index_map['seconds_in_bucket']]
    current_wap = current_data[:, col2index_map['wap']]
    if current_time < 10:
        if len(hist_list) == 0:
            hist_list.append(
                queue.Queue())  # if we directly claim hist_list = [queue.Queue()], it will be a local variable
        else:
            hist_list[0].queue.clear()
        hist_list[0].put(current_wap)
        res = np.zeros(current_wap.shape)
    else:
        hist_list[0].put(current_wap)
        # calculate the past 60s return
        past_wap = hist_list[0].get()
        res = (current_wap - past_wap) / past_wap
    res[np.isnan(res)] = 0  # this is the easiest way to fill nan
    return -res

In [32]:
def past_30s_ret_rank(current_data: dict, hist_list=[]):
    current_data = np.array(list(current_data.values()),
                            dtype=float).T  # this is faster than pd.DataFrame(current_data).values
    current_time = current_data[0, col2index_map['seconds_in_bucket']]
    current_wap = current_data[:, col2index_map['wap']]
    if current_time < 30:
        if current_time == 0:
            if len(hist_list) == 0:
                hist_list.append(queue.Queue())
            else:
                hist_list[0].queue.clear()
        hist_list[0].put(current_wap)
        res = np.zeros(current_wap.shape)
    else:
        hist_list[0].put(current_wap)
        # calculate the past 60s return
        past_wap = hist_list[0].get()
        res = (current_wap - past_wap) / past_wap
    res[np.isnan(res)] = 0  # this is the easiest way to fill nan
    return -stats.rankdata(res, method='min') / (len(res) + 1)

In [33]:
def stock_20s_ret_deviate_from_index(current_data: dict, hist_list=[]):
    current_data = np.array(list(current_data.values()),
                            dtype=float).T  # this is faster than pd.DataFrame(current_data).values
    current_time = current_data[0, col2index_map['seconds_in_bucket']]
    current_wap = current_data[:, col2index_map['wap']]
    if len(hist_list) == 0:
        hist_list.append([
            0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
            0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
            0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
            0.004, 0.004, 0.006, 0.002, 0.002, 0.04, 0.002, 0.002, 0.004, 0.04, 0.002, 0.001,
            0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
            0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
            0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
            0.02, 0.004, 0.006, 0.002, 0.02, 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
            0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
            0.004, 0.006, 0.006, 0.001, 0.04, 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
            0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
            0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
            0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
            0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
            0.04, 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02, 0.004, 0.002, 0.006, 0.02,
            0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
            0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
        ])
    if current_time < 20:
        if current_time == 0:
            if len(hist_list) == 1:
                hist_list.append(queue.Queue())
            else:
                hist_list[1].queue.clear()
        hist_list[1].put(current_wap)
        ret = np.zeros(current_wap.shape)
    else:
        hist_list[1].put(current_wap)
        # calculate the past 60s return
        past_wap = hist_list[1].get()
        ret = (current_wap - past_wap) / past_wap
    # for robustness, if the stock_id is larger than 200, we use the average weight
    weight = [hist_list[0][int(stock_id)] if stock_id <= 200 else np.mean(hist_list[0]) for stock_id in
              current_data[:, col2index_map['stock_id']]]
    synthetic_index = np.sum(np.array(weight, dtype=float) * ret)
    res = ret - synthetic_index
    # if result is nan, we assign res=0
    res[np.isnan(res)] = 0  # this is the easiest way to fill nan
    return -res

In [34]:
def stock_40s_ret_deviate_from_index_rank_hedged_opt(current_data: dict, hist_list=[]):
    current_data = np.array(list(current_data.values()),
                            dtype=float).T  # this is faster than pd.DataFrame(current_data).values
    current_time = current_data[0, col2index_map['seconds_in_bucket']]
    current_wap = current_data[:, col2index_map['wap']]
    if len(hist_list) == 0:
        hist_list.append([
            0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
            0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
            0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
            0.004, 0.004, 0.006, 0.002, 0.002, 0.04, 0.002, 0.002, 0.004, 0.04, 0.002, 0.001,
            0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
            0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
            0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
            0.02, 0.004, 0.006, 0.002, 0.02, 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
            0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
            0.004, 0.006, 0.006, 0.001, 0.04, 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
            0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
            0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
            0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
            0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
            0.04, 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02, 0.004, 0.002, 0.006, 0.02,
            0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
            0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
        ])
    if current_time == 0:
        if len(hist_list) == 1:
            # then we are at the very start, initialize the hist_list with two queues, one for 40s ret one for 20s ret
            hist_list.append(queue.Queue())  # for the 40s ret
            hist_list.append(queue.Queue())  # for the 20s ret
        else:
            # its a new day, we clear the queue
            hist_list[1].queue.clear()
            hist_list[2].queue.clear()
    if current_time < 20:
        hist_list[2].put(current_wap)
        hedge_20s_ret = np.zeros(current_wap.shape)
    else:
        hist_list[2].put(current_wap)
        # calculate the past 60s return
        past_wap = hist_list[2].get()
        hedge_20s_ret = (current_wap - past_wap) / past_wap
    if current_time < 40:
        hist_list[1].put(current_wap)
        ret = np.zeros(current_wap.shape)
    else:
        hist_list[1].put(current_wap)
        # calculate the past 60s return
        past_wap = hist_list[1].get()
        ret = (current_wap - past_wap) / past_wap
    # for robustness, if the stock_id is larger than 200, we use the average weight
    weight = [hist_list[0][int(stock_id)] if stock_id <= 200 else np.mean(hist_list[0]) for stock_id in
              current_data[:, col2index_map['stock_id']]]
    synthetic_index = np.sum(np.array(weight, dtype=float) * ret)
    res = ret - synthetic_index
    hedge_20s_ret = (hedge_20s_ret - np.sum(np.array(weight, dtype=float) * hedge_20s_ret))
    # if result is nan, we assign res=0
    res[np.isnan(res)] = 0  # this is the easiest way to fill nan
    res = stats.rankdata(-res) / len(res)
    res = (res - 0.5) * 2
    res = utils.ols_res(hedge_20s_ret, res, 0.33)
    res[np.isnan(res)] = 0  # this is the easiest way to fill nan

    return res

In [35]:
def stock_40s_ret_deviate_from_index_rank(current_data: dict, hist_list=[]):
    current_data = np.array(list(current_data.values()),
                            dtype=float).T  # this is faster than pd.DataFrame(current_data).values
    current_time = current_data[0, col2index_map['seconds_in_bucket']]
    current_wap = current_data[:, col2index_map['wap']]
    if len(hist_list) == 0:
        hist_list.append([
            0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
            0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
            0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
            0.004, 0.004, 0.006, 0.002, 0.002, 0.04, 0.002, 0.002, 0.004, 0.04, 0.002, 0.001,
            0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
            0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
            0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
            0.02, 0.004, 0.006, 0.002, 0.02, 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
            0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
            0.004, 0.006, 0.006, 0.001, 0.04, 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
            0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
            0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
            0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
            0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
            0.04, 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02, 0.004, 0.002, 0.006, 0.02,
            0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
            0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
        ])
    if current_time < 40:
        if current_time == 0:
            if len(hist_list) == 1:
                hist_list.append(queue.Queue())
            else:
                hist_list[1].queue.clear()
        hist_list[1].put(current_wap)
        ret = np.zeros(current_wap.shape)
    else:
        hist_list[1].put(current_wap)
        # calculate the past 60s return
        past_wap = hist_list[1].get()
        ret = (current_wap - past_wap) / past_wap
    # for robustness, if the stock_id is larger than 200, we use the average weight
    weight = [hist_list[0][int(stock_id)] if stock_id <= 200 else np.mean(hist_list[0]) for stock_id in
              current_data[:, col2index_map['stock_id']]]
    synthetic_index = np.sum(np.array(weight, dtype=float) * ret)
    res = ret - synthetic_index
    # if result is nan, we assign res=0
    res[np.isnan(res)] = 0  # this is the easiest way to fill nan
    return stats.rankdata(-res) / len(res)

In [36]:
def triplet_imbalance_bid_ask_wap_signed_pow_opt_hedged(current_data: dict,hist_list=[])->np.ndarray:
    def triplet_imbalance_cal(data,col):
        assert len(col)==3
        temp= np.vstack([data[:,col2index_map[col[0]]],data[:,col2index_map[col[1]]],data[:,col2index_map[col[2]]]])
        max_val = np.max(temp,axis=0)
        min_val = np.min(temp,axis=0)
        mid_val = np.sum(temp,axis=0)-max_val-min_val
        
        # print(max_val.shape)
        return (max_val-mid_val)/(max_val+mid_val+min_val)
    current_data = np.array(list(current_data.values()),
                            dtype=float).T  # this is faster than pd.DataFrame(current_data).values
    res = triplet_imbalance_cal(current_data,['bid_price','ask_price','wap'])
    res = np.sign(res)*np.abs(res)**0.65
    # s1_imbalance = np.sign(current_data[:, col2index_map['ask_size']] - current_data[:, col2index_map['bid_size']])
    # s1_imbalance[np.isnan(s1_imbalance)] = 0  # this is the fastest w to fill nan
    # res = utils.ols_res(s1_imbalance,res,0.5) # The return value MUSE BE a numpy array, with no NaN value
    res[np.isnan(res)] = 0  # this is the fastest way to fill nan
    return res

In [50]:
def price_pressure(current_data: dict, hist_list=[]) -> np.ndarray:
    '''
    This will be the main function to design your factors for the competition. Please
    define only one factor here each time. We provide you with:

    Current_data: a dictionary in the format of {column_name: column_value}, where column_name is from the original
    dataframe

    hist_list: A list for you to save the previous factor values (optional). For instance,
    if you are calculating a 100-day Moving Average (MA), then you can save the first calculated
    MA in hist_list, and then for the next MA calculation, you can use the saved ones.
    '''
    ###################### ADD YOUR CODE HERE FOR FACTORS DESIGN ######################
    # convert the current_data to your choice of numpy or pandas dataframe
    # current_data = pd.DataFrame(current_data)
    current_data = np.array(list(current_data.values()),
                            dtype=float).T  # this is faster than pd.DataFrame(current_data).values
    res = ((current_data[:, col2index_map['bid_size']] - current_data[:, col2index_map['ask_size']]) / \
          (current_data[:, col2index_map['bid_size']] + current_data[:, col2index_map['ask_size']])) * \
          (current_data[:, col2index_map['ask_price']] - current_data[:, col2index_map['bid_price']])
    ############################## NAN/Inf handling ######################################
    # if you have nan in your factor value, please fill it reasonably
    # res = np.nan_to_num(res) # this is slow because it also checks for inf.
    # res = np.where(np.isnan(res), 0, res)  # this is slightly faster than np.nan_to_num
    res[np.isnan(res)] = 0  # this is the fastest way to fill nan
    ############################## END OF YOUR CODE ##############################
    return -res  # The return value MUSE BE a numpy array, with no NaN value
    ####################################################################################


In [51]:
# new_factor_list = ['s1_imbalance_signed_pow_opt',
#                    "past_10s_ret",
#                    "past_30s_ret_rank",
#                    "stock_20s_ret_deviate_from_index",
#                    "stock_40s_ret_deviate_from_index_rank_hedged_opt",
#                    "stock_40s_ret_deviate_from_index_rank", ]  # fine toned factor set
new_factor_list = ['price_pressure']  

### Factor value calculation in train set

In [52]:
new_factors_train = {factor_name: utils.flatten_factor_value(
    factor_design.run_factor_value(df_train_dic_sorted, eval(factor_name), factor_name), factor_name)[factor_name] for
                     factor_name in tqdm(new_factor_list)}

  0%|          | 0/1 [00:00<?, ?it/s]

Number of dates: 401
Start calculating factor price_pressure
Finished calculating factor price_pressure for 0 dates


C:\Users\kevin\AppData\Local\Temp\ipykernel_16656\2557319847.py:18: RuntimeWarning: invalid value encountered in divide
  res = ((current_data[:, col2index_map['bid_size']] - current_data[:, col2index_map['ask_size']]) / \


Finished calculating factor price_pressure for 100 dates
Finished calculating factor price_pressure for 200 dates
Finished calculating factor price_pressure for 300 dates
Finished calculating factor price_pressure for 400 dates
Accepted!!: Used 14.43 seconds for calculation factors. The limit is 300 seconds.


100%|██████████| 1/1 [00:15<00:00, 15.10s/it]


### Factor value calculation in test set

In [53]:
new_factors_test = {factor_name: utils.flatten_factor_value(
    factor_design.run_factor_value(df_test_dic_sorted, eval(factor_name), factor_name), factor_name)[factor_name] for
                    factor_name in tqdm(new_factor_list)}

  0%|          | 0/1 [00:00<?, ?it/s]

Number of dates: 80
Start calculating factor price_pressure


C:\Users\kevin\AppData\Local\Temp\ipykernel_16656\2557319847.py:18: RuntimeWarning: invalid value encountered in divide
  res = ((current_data[:, col2index_map['bid_size']] - current_data[:, col2index_map['ask_size']]) / \
100%|██████████| 1/1 [00:02<00:00,  2.33s/it]

Finished calculating factor price_pressure for 480 dates
Accepted!!: Used 2.21 seconds for calculation factors. The limit is 300 seconds.


## Factor Backtesting

In [54]:
demo_backtest = factor_backtest.Factor_Backtest(existed_factors=existed_factor_train, testing_factors=new_factors_train,
                                                factor_performance=factor_performance_train)  # just a formality 

In [41]:
print(existed_factor_train.keys())

dict_keys(['origin_seconds_in_bucket', 'origin_imbalance_size', 'origin_imbalance_buy_sell_flag', 'origin_reference_price', 'origin_matched_size', 'origin_far_price', 'origin_near_price', 'origin_bid_price', 'origin_bid_size', 'origin_ask_price', 'origin_ask_size', 'origin_wap', 's1_imbalance_signed_pow_opt', 'stock_20s_ret_deviate_from_index', 'stock_40s_ret_deviate_from_index_rank_hedged_opt'])


In [55]:
demo_backtest.validate_new_factors(new_factors_train, new_factors_test)


_______________________________________Factor Performance_______________________________________________________
Factor price_pressure PASSED in-sample performance check with correlation coefficient 0.15130224198627285

___________________________________Factor pairwise Correlation__________________________________________________
Factor price_pressure passed in-sample pairwise correlation check with all existed factors
______________________________________________________________________________________________
Factor price_pressure passed In-sample tests
Out of sample correlation coefficient for factor price_pressure is 0.10690736685966976
Factor price_pressure passed out-of-sample performance 
 in-sample corr = 0.15130224198627285 
 out-of-sample corr = 0.10690736685966976
Factor price_pressure passed Out-of-sample tests
Factor price_pressure passed all the tests, add to the factor pool
No temporary files found, creating one


______________________________________Overall Conclusi

In [43]:
existed_factor_test.keys()

dict_keys(['origin_seconds_in_bucket', 'origin_imbalance_size', 'origin_imbalance_buy_sell_flag', 'origin_reference_price', 'origin_matched_size', 'origin_far_price', 'origin_near_price', 'origin_bid_price', 'origin_bid_size', 'origin_ask_price', 'origin_ask_size', 'origin_wap', 's1_imbalance_signed_pow_opt', 'stock_20s_ret_deviate_from_index', 'stock_40s_ret_deviate_from_index_rank_hedged_opt'])

In [44]:
existed_factor_train['stock_20s_ret_deviate_from_index']

[-0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,

In [56]:
existed_factor_test = utils.add_factor_to_existed_test()

matching files: ['./in_house_validation\\test_tmp20231216.json']
Factor price_pressure is a new factor, adding the value
./in_house_validation\test_tmp20231216.json added to existed_factors
storing the updated existed_factors
existed_factors updated, all temporary files deleted


In [46]:
existed_factor_test.keys()

dict_keys(['origin_seconds_in_bucket', 'origin_imbalance_size', 'origin_imbalance_buy_sell_flag', 'origin_reference_price', 'origin_matched_size', 'origin_far_price', 'origin_near_price', 'origin_bid_price', 'origin_bid_size', 'origin_ask_price', 'origin_ask_size', 'origin_wap', 's1_imbalance_signed_pow_opt', 'stock_20s_ret_deviate_from_index', 'stock_40s_ret_deviate_from_index_rank_hedged_opt', 'triplet_imbalance_bid_ask_wap_signed_pow_opt_hedged'])

In [57]:
existed_factor_train = utils.add_factor_to_existed_train()

matching files: ['./in_house_validation\\tmp20231216.json']
Factor price_pressure is a new factor, adding the value
./in_house_validation\tmp20231216.json added to existed_factors
storing the updated existed_factors
existed_factors updated, all temporary files deleted


In [58]:
existed_factor_train.keys()

dict_keys(['origin_seconds_in_bucket', 'origin_imbalance_size', 'origin_imbalance_buy_sell_flag', 'origin_reference_price', 'origin_matched_size', 'origin_far_price', 'origin_near_price', 'origin_bid_price', 'origin_bid_size', 'origin_ask_price', 'origin_ask_size', 'origin_wap', 's1_imbalance_signed_pow_opt', 'stock_20s_ret_deviate_from_index', 'stock_40s_ret_deviate_from_index_rank_hedged_opt', 'triplet_imbalance_bid_ask_wap_signed_pow_opt_hedged', 'price_pressure'])